In [1]:
# Step 1: Imports
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint


In [2]:
# Step 2: Paths and Parameters
train_data_dir = 'train/train'
test_data_dir = 'test_images/Kaggle_test_images'
img_size = (32, 32)
batch_size = 32  # Smaller batch = better accuracy


In [3]:
# Step 3: Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 72000 images belonging to 10 classes.
Found 18000 images belonging to 10 classes.


In [4]:
# Step 4: Custom CNN Model (Deeper and Better)
model = Sequential([
    Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.3),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.4),

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


C:\Users\ashar\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Step 5: Compile and Train for 40 Epochs
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    callbacks=callbacks
)


C:\Users\ashar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 800s 352ms/step - accuracy: 0.2481 - loss: 2.4031 - val_accuracy: 0.4024 - val_loss: 1.6901 - learning_rate: 5.0000e-04
Epoch 2/40
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 751s 334ms/step - accuracy: 0.3752 - loss: 1.6962 - val_accuracy: 0.3818 - val_loss: 1.6802 - learning_rate: 5.0000e-04
Epoch 3/40
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 760s 338ms/step - accuracy: 0.4448 - loss: 1.5062 - val_accuracy: 0.4978 - val_loss: 1.3651 - learning_rate: 5.0000e-04
Epoch 4/40
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 761s 338ms/step - accuracy: 0.4840 - loss: 1.4085 - val_accuracy: 0.5656 - val_loss: 1.2059 - learning_rate: 5.0000e-04
Epoch 5/40
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 771s 342ms/step - accuracy: 0.5155 - loss: 1.3268 - val_accuracy: 0.5997 - val_loss: 1.1181 - learning_rate: 5.0000e-04
Epoch 6/40
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 769s 342ms/step - accuracy: 0.5403 - loss: 1.2671 - val_accuracy: 0.6032 - val_loss: 1.1132 - learning_rate: 5.0000e-04
Epoch 7/40
2250/2250 ━━━━━━━

In [6]:
# Step 6: Load Best Model
model = load_model("best_model.keras")

val_loss, val_acc = model.evaluate(val_gen)
print(f"✅ Final Validation Accuracy: {val_acc:.4f}")


563/563 ━━━━━━━━━━━━━━━━━━━━ 55s 97ms/step - accuracy: 0.7922 - loss: 0.6077
✅ Final Validation Accuracy: 0.7960


In [7]:
# Step 7: Load Test Images
test_images = []
test_ids = []

for filename in sorted(os.listdir(test_data_dir)):
    path = os.path.join(test_data_dir, filename)
    img = cv2.imread(path)
    img = cv2.resize(img, img_size)
    img = img.astype('float32') / 255.0
    test_images.append(img)
    test_ids.append(filename)

test_images = np.array(test_images)
print("Test images loaded:", test_images.shape)


Test images loaded: (10000, 32, 32, 3)


In [8]:
# Step 8: Predict on Test Images
preds = model.predict(test_images)
pred_labels = np.argmax(preds, axis=1)


313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 77ms/step


In [9]:
# Step 9: Save Prediction CSV
submission = pd.DataFrame({
    "id": test_ids,
    "label": pred_labels
})
submission.to_csv("Ashar_CV5_submission.csv", index=False)
print("🎯 Submission saved as: Ashar_CV5_submission.csv")


🎯 Submission saved as: Ashar_CV5_submission.csv
